https://www.kaggle.com/code/mehmetisik/content-based-recommendation

In [ ]:
import os
# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = os.getcwd().replace("\\", "/")

datasets_path = "/datasets/"
path_absolute = path_absolute+datasets_path

![CBR](https://miro.medium.com/v2/resize:fit:1400/1*H_MMnrpLQrqTSJHdDOCMoA.png)

# What is Content Based Recommendation

Content-based recommendation, also known as content-based filtering, is a type of system or algorithm that provides recommendations to a user based on their interests and preferences. Those with such recommendation systems analyze the user's past preferences and likes, and suggest new items based on similar content.

Content-based recommendation analyzes the content of items and determines the ones that are suitable for the user based on similarity criteria. For example, when making a movie recommendation, the system can take into account the genres, actors, directors, and other features of the movies the user has liked or watched. Based on this information, the system suggests other movies with similar characteristics.

This recommendation system can utilize text analysis, tagging, categorization, or other content features along with the user profile or history to better understand the user's preferences. For instance, when making a music recommendation, the system can analyze features such as genres, instruments, tempo, and rhythm.

Content-based recommendation systems can be effective in providing personalized recommendations based on user preferences. The recommended items based on the user's past data can capture their interest and provide a better user experience.

# Business Problem
To recommend movies similar to the movies that a person who comes to our site to watch movies.

# Road Map

- 1. Creating the **TF-IDF Matrix**
- 2. Creation of **Cosine Similarity Matrix**
- 3. Making Recommendations Based on Similarities
- 4. Preparation of the Study Script


In [ ]:
# import Required Libraries

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
# Loading the Data Set
# df = pd.read_csv(path_absolute+"movies_metadata.csv")


In [ ]:
df = pd.read_csv(path_absolute+"df_mezclado_tags_ratings_movies_links_genTags.csv")

In [ ]:
tamano_muestra = 70000
df = df.sample(n=tamano_muestra, random_state=42)

In [ ]:
df.head()

In [ ]:
# Eliminar las columnas imdbId y tmdbId
# Renombrar las columnas tal que: 'timestamp_valoraciones': 'timestamp_scr' y 'timestamp_etiquetas': 'timestamp_tags'
#Mover las columnas de rating y relevance al final y la de tag_etiquetas_genómicas despues de tag_df_mezclado...

# Inserta la columna 'rating' en la última posición del DataFrame
df.insert(len(df.columns)-1, 'rating', df.pop('rating'))
df.insert(len(df.columns)-2, 'relevance', df.pop('relevance'))
df.insert(5, 'tag_etiquetas_genómicas', df.pop('tag_etiquetas_genómicas'))

df.rename(columns={'timestamp_valoraciones': 'timestamp_rt', 'timestamp_etiquetas': 'timestamp_tags', 'tag_df_mezclado_tags_ratings_movies_links_genMov': 'tag_by_user', 'tag_etiquetas_genómicas': 'gen_tag'}, inplace=True)

columnas_a_eliminar = ['imdbId', 'tmdbId']
# Elimina las columnas especificadas del DataFrame 'data'
df = df.drop(columnas_a_eliminar, axis=1)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = df.isnull().sum()

# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = df.isnull().sum().sum()

# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)

# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

In [ ]:
# Seleccionar las filas con valores nulos en la columna deseada
filas_con_nulos = df[df['tag_by_user'].isnull()]

# Mostrar las filas con valores nulos
print("Filas con valores nulos en la columna tag:")
print(filas_con_nulos)

# Eliminar las filas con valores nulos en la columna deseada
df = df.dropna(subset=['tag_by_user'])
print("DataFrame después de eliminar filas con valores nulos:")
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = df.isnull().sum()
# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = df.isnull().sum().sum()
# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)
# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

In [ ]:
df.shape

In [ ]:
content_df = df[['title', 'genres', 'tag_by_user', 'gen_tag', 'relevance', 'rating']]

In [ ]:
content_df = content_df.dropna(subset=['tag_by_user'])

In [ ]:
import re
import string

def remove_bars(text):
    # Eliminar barras y dejar palabras separadas por un espacio
    cleaned = re.sub(r'\|', ' ', text)
    return cleaned

def separate(text):
    clean_text = []
    for t in text.split(','):
        cleaned = re.sub('\(.*\)', '', t) # Remove text inside parentheses
        cleaned = cleaned.translate(str.maketrans('','', string.digits))
        cleaned = cleaned.replace(' ', '')
        cleaned = cleaned.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text.append(cleaned)
    return ' '.join(clean_text)

def remove_punc(text):
    try:
        cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text = cleaned.translate(str.maketrans('','', string.digits))
    except Exception as e:
        print(f"ERROR -----------------------------------------> {e} AND {text}")
    return clean_text

In [ ]:
# Ejemplo de uso: Pixar Animation Studios Warner Bros., Lancaster Gate
text_with_bars = "oldie but goodie"

# text_with_bars = "Pixar Animation Studios"

# text_with_bars = "Warner Bros., Lancaster Gate"

cleaned_text = remove_punc(text_with_bars)
print(cleaned_text)

In [ ]:
content_df['genres'] = (content_df['genres'].apply(remove_bars)).apply(remove_punc)
content_df['tag_by_user'] = content_df['tag_by_user'].apply(remove_punc)
# content_df.set_index('title', inplace=True)
content_df = content_df.reset_index()

In [ ]:
content_df.head()

In [ ]:
content_df_aux = content_df.copy()
content_df_aux = content_df_aux[['genres', 'tag_by_user', 'gen_tag']]
content_df_aux['bag_of_words'] = ''
content_df_aux.loc[:, 'bag_of_words'] = content_df_aux.loc[:, content_df_aux.columns[0:]].apply(lambda x: ' '.join(x), axis=1)
# content_df.set_index('original_title', inplace=True)
columnas_a_eliminar = ['tag_by_user', 'gen_tag']
# Elimina las columnas especificadas del DataFrame 'data'
content_df = content_df.drop(columnas_a_eliminar, axis=1)

content_df['genres'] = content_df_aux['bag_of_words']
content_df.rename(columns={'genres': 'bag_of_words'}, inplace=True)

content_df.head()

# 1. Creating the TF-IDF Matrix

In [ ]:
df = content_df.copy()

In [ ]:
df["bag_of_words"].head()

In [ ]:
df["bag_of_words"].isnull().sum()

In [ ]:
# Convertir la colección de textos (en este caso, la bolsa de palabras) 
# en una matriz TF-IDF.

# Vamos a eliminar las construcciones como a, an, the, and, 
# pero que no tienen sentido para nosotros de nuestros DataFrames.
tfidf = TfidfVectorizer(stop_words="english")

In [ ]:
# Rellenemos el valor nulo de la variable cin bag_of_words con nada para evitar errores en los siguientes pasos
df['bag_of_words'] = df['bag_of_words'].fillna('')

In [ ]:
df["bag_of_words"].isnull().sum()

In [ ]:
# ajustar y transformar según el objeto tfidf
# Los de las filas son textos 'bolsa de palabras'. Los de las columnas son palabras únicas.

# Crear la matriz TF-IDF (Term Frequency-Inverse Document Frequency) a partir de la columna 
# 'bag_of_words' del conjunto de datos. La matriz TF-IDF es una representación numérica 
# de los documentos en función de la frecuencia de las palabras que contienen y de 
# su importancia en el conjunto de documentos.
tfidf_matrix = tfidf.fit_transform(df['bag_of_words'])

In [ ]:
# Cada fila representa un documento (en este caso, una película) y 
# cada columna representa una palabra única en el conjunto de datos.

# Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada documento. 
tfidf_matrix.shape

In [ ]:
#Si queremos ver todas las palabras únicas de las columnas

# Filas: Cada fila de la matriz corresponde a una película en el conjunto de datos. 
# Por ejemplo, si tienes 10,000 películas, habrá 10,000 filas en la matriz.

# Columnas: Cada columna representa una palabra única en el conjunto de datos. 
# La cantidad de columnas es igual al número total de palabras únicas en todas 
# las bolsas de palabras de películas. 

# Valores: Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada película. 
# Estas puntuaciones miden la importancia relativa de una palabra en un documento en comparación 
# con su frecuencia en el conjunto de documentos. Valores más altos indican que la palabra 
# es más importante en el contexto de ese documento.

tfidf.get_feature_names_out()


In [ ]:
# tfidf scores
tfidf_matrix.toarray()

# 2. Creation of Cosine Similarity Matrix

In [ ]:
# calcula la similitud coseno entre todas las filas de la matriz TF-IDF. 
# La similitud coseno mide el coseno del ángulo entre dos vectores y 
# proporciona una medida de cuán similares son dos documentos.

# Calcula cos sim para todos los pares de documentos posibles uno por uno. En la matriz cosine_sim, 
# cada película tiene similitudes entre sí

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# cosine_sim[i, j] representa la similitud coseno entre la película con el índice i y 
# la película con el índice j en el conjunto de datos.

cosine_sim.shape

In [ ]:
cosine_sim

In [ ]:
# Para ver en qué se parece la película del índice 1 a todas las demás películas

cosine_sim[1]

In [ ]:
cosine_sim[1].shape

# 3. Making Recommendations Based on Similarities

In [ ]:
# Vamos a crear una serie pd de índices y nombres de películas
# se crea una Serie de Pandas llamada indices donde los índices son
# los títulos de las películas y los valores son los índices originales del DataFrame df.
indices = pd.Series(df.index, index=df['title'])

In [ ]:
indices.head()

In [ ]:
indices.shape

In [ ]:
# contemos la información del índice de las películas y simplifiquemos las películas más repetitivas a las más recientes

indices.index.value_counts().head()

In [ ]:
# Se eliminan las películas duplicadas del índice, conservando solo la última aparición de cada película. 
# Esto se hace para evitar ambigüedades y asegurar que cada película tenga un índice único.
indices = indices[~indices.index.duplicated(keep='last')]

In [ ]:
indices.index.value_counts().head()

In [ ]:
indices.shape

In [ ]:
indices["Matrix, The (1999)"]

In [ ]:
indices["Wolf of Wall Street, The (2013)"]

In [ ]:
# Asigno el índice de la película "Sherlock Holmes" a la variable
movie_title = "Wolf of Wall Street, The (2013)"
movie_index = indices[movie_title]

In [ ]:
cosine_sim[movie_index]

In [ ]:
cosine_sim[movie_index].shape

In [ ]:
# Veamos los Smilarity Scores que expresan las similitudes entre la película "Wolf of Wall Street, The (2013)" y otras películas
similarity_scores = pd.DataFrame(cosine_sim[movie_index],
                                 columns=["score"])

In [ ]:
# Las similitudes entre la película 'Sherlock Holmes' y el resto de películas

similarity_scores.head()

In [ ]:
similarity_scores.shape

In [ ]:
movie_indices_sort = similarity_scores.sort_values("score", ascending=False).index

In [ ]:
import pandas as pd

# Supongamos que tu DataFrame se llama df y la Serie de índices es index_series
# index_series = Int64Index([9859, 3228, 1656, 8951, 6811, 998, 5030, 6972, 8591, 4820, ...])

# Paso 1
duplicated_titles = df.loc[movie_indices_sort, 'title'].duplicated(keep='first')

# Paso 2
indices_sin_duplicados = movie_indices_sort[~duplicated_titles]

# Paso 3
filas_sin_duplicados = df.loc[indices_sin_duplicados]

# Ahora filas_sin_duplicados contiene las filas correspondientes a los índices
# en index_series, eliminando las filas con títulos duplicados, manteniendo la primera ocurrencia.

# Supongamos que el string que deseas excluir es 'The Promise'
string_a_excluir = movie_title

# Filtrar las filas que no contienen el string a excluir en la columna 'title'
filas_sin_duplicados_y_exclusion = df.loc[indices_sin_duplicados]
filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != string_a_excluir]

In [ ]:
# Supongamos que el string que deseas excluir es 'The Promise'
string_a_excluir = 'Wolf of Wall Street, The (2013)'

# Filtrar las filas que no contienen el string a excluir en la columna 'title'
filas_sin_duplicados_y_exclusion = df.loc[indices_sin_duplicados]
filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != string_a_excluir]


In [ ]:
filas_sin_duplicados_y_exclusion.head()

--------------------------------------------------------------------------

In [ ]:
# Vamos a listar las puntuaciones de similitud de la película 'Sherlock Holmes' en orden descendente. Empieza por 1 porque es la primera película.

# movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index

In [ ]:
# Go to the indexes we selected in our first data set

# df['title'].iloc[movie_indices]

# 4. Preparation of the Study Script

In [ ]:
# def content_based_recommender(title, cosine_sim, dataframe):
#     # create indexes
#     indices = pd.Series(dataframe.index, index=dataframe['title'])
#     indices = indices[~indices.index.duplicated(keep='last')]
#     # capturing index of title
#     movie_index = indices[title]
#     # calculate similarity scores based on title
#     similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
#     # Bringing the top 10 movies except for itself
#     movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
#     return dataframe['title'].iloc[movie_indices]

In [ ]:
def content_based_recommender(title, cosine_sim, dataframe):
    # create indexes
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    # capturing index of title
    movie_index = indices[title]
    # calculate similarity scores based on title
    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    
    # Bringing the top 10 movies except for itself
    movie_indices_sort = similarity_scores.sort_values("score", ascending=False).index
    duplicated_titles = dataframe.loc[movie_indices_sort, 'title'].duplicated(keep='first')
    indices_sin_duplicados = movie_indices_sort[~duplicated_titles]
    filas_sin_duplicados_y_exclusion = dataframe.loc[indices_sin_duplicados]
    filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != title]
    return filas_sin_duplicados_y_exclusion['title'].head(10)

In [ ]:
content_based_recommender("Wolf of Wall Street, The (2013)", cosine_sim, df)

In [ ]:
content_based_recommender("Matrix, The (1999)", cosine_sim, df)

In [ ]:
content_based_recommender("Star Wars: Episode IV - A New Hope (1977)", cosine_sim, df)

In [ ]:
content_based_recommender('Pulp Fiction (1994)', cosine_sim, df)

In [ ]:
def calculate_cosine_sim(dataframe):
    tfidf = TfidfVectorizer(stop_words='english')
    dataframe['bag_of_words'] = dataframe['bag_of_words'].fillna('')
    tfidf_matrix = tfidf.fit_transform(dataframe['bag_of_words'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

In [ ]:
# cosine_sim = calculate_cosine_sim(df)
# content_based_recommender('Pulp Fiction (1994)', cosine_sim, df)